```{contents}
```
## Immutable State Transitions


In LangGraph, **Immutable State Transitions** refer to the principle that **state is never modified in-place**; instead, each node produces **new state objects** that replace or merge with the existing state. This functional programming paradigm ensures **predictability, reproducibility, safe concurrency, and reliable checkpointing**.

---

### **1. Motivation and Intuition**

Traditional stateful systems often use **mutable state**, where data structures are modified directly:

```python
# Mutable approach (NOT LangGraph)
state["messages"].append(new_message)  # modifies in-place
state["counter"] += 1                   # mutates existing value
```

This creates problems:
- **Race conditions** in concurrent execution
- **Checkpoint corruption** when state changes mid-save
- **Hard-to-debug behavior** from hidden mutations
- **Irreproducible runs** due to mutation order

LangGraph uses **immutable transitions** instead:

| Mutable Approach       | Immutable Approach                     |
| ---------------------- | -------------------------------------- |
| Modify state directly  | Return new state                       |
| Unpredictable ordering | Deterministic updates                  |
| Hard to replay         | Trivial to replay                      |
| Concurrency issues     | Thread-safe by default                 |
| Checkpoint drift       | Perfect checkpoint consistency         |

---

### **2. Conceptual Model**

```
State(t)  ────→  [  Node  ]  ────→  State(t+1)
  (input)       (pure function)      (new output)

State(t) remains unchanged
State(t+1) = merge(State(t), node_output)
```

Each transition creates a **new version** of the state.

---

### **3. How Immutable State Transitions Work**

#### **Core Mechanism**

Nodes receive the current state and **return updates**:

```python
from typing import TypedDict

class GraphState(TypedDict):
    messages: list[str]
    count: int
    results: dict

def my_node(state: GraphState) -> dict:
    # State parameter is READ-ONLY
    # DO NOT modify state directly
    
    # Instead, return new values
    return {
        "messages": state["messages"] + ["new message"],  # new list
        "count": state["count"] + 1                        # new value
    }
```

LangGraph then **merges** the returned dictionary into the state.

---

### **4. State Update Strategies**

LangGraph supports different merge strategies through **reducers**:

#### **Default: Replace**
```python
class State(TypedDict):
    result: str

def node(state):
    return {"result": "new value"}  # replaces old value
```

#### **Append to List**
```python
from langgraph.graph import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]  # special reducer

def node(state):
    return {"messages": [new_message]}  # appends, doesn't replace
```

#### **Custom Reducer**
```python
def merge_dicts(existing: dict, new: dict) -> dict:
    return {**existing, **new}  # immutable merge

class State(TypedDict):
    data: Annotated[dict, merge_dicts]
```

All strategies produce **new state objects**.

---

### **5. Why Immutability Matters**

| Benefit              | How Immutability Helps                            |
| -------------------- | ------------------------------------------------- |
| **Reproducibility**  | Same input → same output, always                  |
| **Time Travel**      | Can restore any past state exactly                |
| **Debugging**        | Every state transition is traceable               |
| **Concurrency**      | No race conditions, safe parallel execution       |
| **Checkpointing**    | State snapshots are consistent                    |
| **Testing**          | Pure functions are easy to test                   |
| **Reasoning**        | Easier to understand data flow                    |

---

### **6. Immutable vs Mutable State**

| Feature           | Immutable (LangGraph)       | Mutable (Traditional)  |
| ----------------- | --------------------------- | ---------------------- |
| Node function     | Returns new state           | Modifies state         |
| Safety            | Thread-safe                 | Requires locking       |
| Replay            | Exact replay possible       | Cannot replay reliably |
| Debugging         | Clear state history         | Hidden mutations       |
| Performance       | Copy overhead (minimal)     | No copy overhead       |
| Complexity        | Simpler reasoning           | Complex side effects   |

---

### **7. Example: Immutable Message Accumulation**

```python
from typing import Annotated
from langgraph.graph import StateGraph, add_messages

class ChatState(TypedDict):
    messages: Annotated[list, add_messages]
    analysis: dict

def agent_node(state: ChatState) -> dict:
    # Read current state (immutable)
    current_messages = state["messages"]
    
    # Compute new message (local computation)
    response = llm.invoke(current_messages)
    
    # Return NEW state update (immutable)
    return {
        "messages": [response],  # add_messages will append immutably
        "analysis": {"tokens": len(response.content)}
    }

# LangGraph creates: State(new) = merge(State(old), agent_node(State(old)))
```

**Key Point**: `current_messages` is never modified. New messages are returned.

---

### **8. Immutability in Cyclic Graphs**

In loops, immutability prevents **state corruption**:

```python
def loop_node(state):
    iteration = state.get("iteration", 0)
    
    # WRONG: state["iteration"] += 1  # mutation!
    
    # CORRECT: return new value
    return {
        "iteration": iteration + 1,
        "data": process(state["data"])  # new processed data
    }
```

Each cycle iteration produces a **clean, new state version**.

---

### **9. Checkpoint Consistency**

Immutability guarantees checkpoints are **atomic snapshots**:

```
Checkpoint 1: State(v1)
    ↓
[Node executes]
    ↓
Checkpoint 2: State(v2) = merge(State(v1), updates)
```

Because `State(v1)` is immutable, it remains valid even after checkpoint 2 is created.

---

### **10. Performance Considerations**

**Q: Doesn't copying state on every transition hurt performance?**

**A: No, for several reasons:**

1. **Structural Sharing**: Python and LangGraph reuse unchanged parts
   ```python
   old_state = {"messages": [1, 2, 3], "config": {...}}
   new_state = {**old_state, "messages": old_state["messages"] + [4]}
   # "config" is shared, not copied
   ```

2. **Shallow Copies**: Only top-level structure is copied
3. **Lazy Evaluation**: Some frameworks defer copies until needed
4. **Trade-off**: Small copy cost << debugging/concurrency benefits

---

### **11. Anti-Patterns to Avoid**

| Anti-Pattern                  | Why Bad                          | Correct Approach          |
| ----------------------------- | -------------------------------- | ------------------------- |
| `state["x"] = y`              | Mutates state                    | `return {"x": y}`         |
| `state["list"].append(item)`  | Mutates list                     | `return {"list": [...] }` |
| Modifying nested objects      | Hidden mutation                  | Deep copy + modify        |
| Relying on mutation order     | Non-deterministic                | Pure functions            |
| Side effects in state updates | Violates functional purity       | Keep side effects local   |

---

### **12. Mental Model**

> **Immutable State = Git Commits**  
> Each state transition is like a git commit: creates a new version, old versions remain intact.

```
State(v1) → [Node A] → State(v2) → [Node B] → State(v3)
  ↓                      ↓                      ↓
checkpoint 1          checkpoint 2          checkpoint 3
```

You can always return to any checkpoint.

---

### **13. When to Use Immutable Patterns**

**Always** in LangGraph, but especially critical for:

- **Multi-agent systems**: Prevents agents from interfering with each other
- **Human-in-the-loop**: State must be frozen while awaiting input
- **Long-running workflows**: Checkpoints must remain consistent
- **Parallel execution**: Concurrent nodes cannot corrupt shared state
- **Production systems**: Reproducibility and debugging are essential

---

### **14. Advanced: Custom Immutable Reducers**

For complex state types, define custom immutable reducers:

```python
from typing import Annotated

def immutable_merge_scores(old: dict, new: dict) -> dict:
    """Merge score dictionaries immutably."""
    return {**old, **new}  # creates new dict

class State(TypedDict):
    scores: Annotated[dict, immutable_merge_scores]

def scorer(state):
    return {"scores": {"quality": 0.95}}  # merged immutably
```

---

### **15. Functional Purity and State Transitions**

Immutability enables **pure functions**:

```python
def pure_node(state: State) -> dict:
    # Given the same input state, ALWAYS returns the same output
    # No side effects (except I/O like LLM calls, which are external)
    
    result = deterministic_transform(state["input"])
    return {"output": result}
```

**Benefits**:
- Testable: `assert pure_node(test_state) == expected_output`
- Cacheable: Same input → same output
- Parallelizable: No shared mutable state

---

### **16. Immutability and Human-in-the-Loop**

When execution pauses for human input, immutability ensures **state integrity**:

```python
def before_human_node(state):
    # State is checkpointed here (immutably)
    return {"status": "awaiting_human"}

# [Execution pauses]
# State remains unchanged while waiting
# User provides input

def after_human_node(state):
    # State is exactly as it was before pause
    return {"status": "processing", "user_input": get_input()}
```

---

### **17. Key Design Principles**

> **1. Treat state as read-only input**  
> **2. Return only what changed**  
> **3. Use reducers for complex merge logic**  
> **4. Never mutate nested structures**  
> **5. Embrace functional purity**

Following these principles yields **reliable, scalable, maintainable LangGraph applications**.

---

### **18. Comparison with Other Paradigms**

| Paradigm            | State Handling    | LangGraph Alignment |
| ------------------- | ----------------- | ------------------- |
| Functional          | Immutable         | ✅ Perfect match    |
| Object-Oriented     | Often mutable     | ⚠️ Requires care    |
| Reactive (Rx)       | Immutable streams | ✅ Compatible       |
| Event Sourcing      | Immutable events  | ✅ Similar model    |
| Traditional Agents  | Mutable blackboard | ❌ Incompatible     |

LangGraph adopts **functional + event sourcing** principles.

---

### **Demonstration**

### **Example 1: Basic Immutable State Update**

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

# Define state schema
class CounterState(TypedDict):
    count: int
    message: str

# Node that demonstrates immutable state update
def increment_node(state: CounterState) -> dict:
    """
    Demonstrates immutable state transition.
    NEVER modifies the input state directly.
    """
    # Read from state (immutable access)
    current_count = state["count"]
    
    # Compute new values locally
    new_count = current_count + 1
    new_message = f"Count is now {new_count}"
    
    # Return NEW state values (immutable update)
    return {
        "count": new_count,
        "message": new_message
    }

# Build graph
graph = StateGraph(CounterState)
graph.add_node("increment", increment_node)
graph.set_entry_point("increment")
graph.add_edge("increment", END)

app = graph.compile()

# Execute with initial state
initial_state = {"count": 0, "message": "Starting"}
print("Initial state:", initial_state)

result = app.invoke(initial_state)
print("\nAfter transition:", result)
print("\nOriginal state unchanged:", initial_state)

Initial state: {'count': 0, 'message': 'Starting'}

After transition: {'count': 1, 'message': 'Count is now 1'}

Original state unchanged: {'count': 0, 'message': 'Starting'}


### **Example 2: Immutable List Operations with Reducers**

In [ ]:
from typing import Annotated
from langgraph.graph import StateGraph, END

# Custom reducer for immutable list append
def append_immutably(existing: list, new: list) -> list:
    """Immutably appends new items to existing list."""
    return existing + new  # Creates new list, doesn't modify existing

class LogState(TypedDict):
    logs: Annotated[list[str], append_immutably]
    step: int

def log_step_1(state: LogState) -> dict:
    """First processing step."""
    # WRONG: state["logs"].append("Step 1")  # This would mutate!
    
    # CORRECT: Return new items to append
    return {
        "logs": ["Started processing"],
        "step": 1
    }

def log_step_2(state: LogState) -> dict:
    """Second processing step."""
    return {
        "logs": [f"Completed step {state['step']}", "Moving to step 2"],
        "step": 2
    }

def log_step_3(state: LogState) -> dict:
    """Final step."""
    return {
        "logs": [f"Completed step {state['step']}", "Finished all steps"],
        "step": 3
    }

# Build graph
graph = StateGraph(LogState)
graph.add_node("step1", log_step_1)
graph.add_node("step2", log_step_2)
graph.add_node("step3", log_step_3)

graph.set_entry_point("step1")
graph.add_edge("step1", "step2")
graph.add_edge("step2", "step3")
graph.add_edge("step3", END)

app = graph.compile()

# Execute
initial_state = {"logs": [], "step": 0}
result = app.invoke(initial_state)

print("Final logs:")
for i, log in enumerate(result["logs"], 1):
    print(f"  {i}. {log}")
print(f"\nFinal step: {result['step']}")
print(f"\nOriginal state still intact: {initial_state}")

### **Example 3: Immutable Nested Data Structures**

In [2]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
import copy

class NestedState(TypedDict):
    user: dict
    settings: dict
    metrics: dict

def update_user_profile(state: NestedState) -> dict:
    """
    Demonstrates immutable updates to nested dictionaries.
    CRITICAL: Must create new dictionary, not modify existing one.
    """
    # WRONG APPROACH (mutates state):
    # state["user"]["name"] = "Alice"  # DON'T DO THIS!
    
    # CORRECT APPROACH 1: Spread operator
    new_user = {
        **state["user"],  # Copy existing fields
        "name": "Alice",  # Update specific field
        "updated": True
    }
    
    # CORRECT APPROACH 2: For deep nested structures
    new_settings = copy.deepcopy(state["settings"])
    new_settings["theme"] = "dark"  # Now safe to modify (it's a copy)
    
    return {
        "user": new_user,      # New dictionary
        "settings": new_settings  # New dictionary
    }

def calculate_metrics(state: NestedState) -> dict:
    """Calculate metrics immutably."""
    # Create completely new metrics dictionary
    new_metrics = {
        "user_updates": state["metrics"].get("user_updates", 0) + 1,
        "last_update": "2025-12-28",
        "active": True
    }
    
    return {"metrics": new_metrics}

# Build graph
graph = StateGraph(NestedState)
graph.add_node("update_profile", update_user_profile)
graph.add_node("calculate", calculate_metrics)

graph.set_entry_point("update_profile")
graph.add_edge("update_profile", "calculate")
graph.add_edge("calculate", END)

app = graph.compile()

# Execute
initial_state = {
    "user": {"name": "Unknown", "id": 123},
    "settings": {"theme": "light", "notifications": True},
    "metrics": {"user_updates": 0}
}

print("Initial state:")
print(f"  User: {initial_state['user']}")
print(f"  Settings: {initial_state['settings']}")
print(f"  Metrics: {initial_state['metrics']}")

result = app.invoke(initial_state)

print("\nFinal state:")
print(f"  User: {result['user']}")
print(f"  Settings: {result['settings']}")
print(f"  Metrics: {result['metrics']}")

print("\nOriginal state unchanged:")
print(f"  Original user: {initial_state['user']}")
print(f"  Original settings: {initial_state['settings']}")

Initial state:
  User: {'name': 'Unknown', 'id': 123}
  Settings: {'theme': 'light', 'notifications': True}
  Metrics: {'user_updates': 0}

Final state:
  User: {'name': 'Alice', 'id': 123, 'updated': True}
  Settings: {'theme': 'dark', 'notifications': True}
  Metrics: {'user_updates': 1, 'last_update': '2025-12-28', 'active': True}

Original state unchanged:
  Original user: {'name': 'Unknown', 'id': 123}
  Original settings: {'theme': 'light', 'notifications': True}


### **Example 4: Checkpoint Consistency with Immutability**

In [3]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

class CheckpointState(TypedDict):
    version: int
    data: str
    history: list[str]

def process_step_1(state: CheckpointState) -> dict:
    """First processing step - demonstrates checkpoint at version 1."""
    return {
        "version": state["version"] + 1,
        "data": "Processed in step 1",
        "history": state["history"] + ["Step 1 completed"]
    }

def process_step_2(state: CheckpointState) -> dict:
    """Second processing step - demonstrates checkpoint at version 2."""
    return {
        "version": state["version"] + 1,
        "data": "Processed in step 2",
        "history": state["history"] + ["Step 2 completed"]
    }

def process_step_3(state: CheckpointState) -> dict:
    """Third processing step - demonstrates checkpoint at version 3."""
    return {
        "version": state["version"] + 1,
        "data": "Processed in step 3",
        "history": state["history"] + ["Step 3 completed"]
    }

# Build graph with checkpointing
graph = StateGraph(CheckpointState)
graph.add_node("step1", process_step_1)
graph.add_node("step2", process_step_2)
graph.add_node("step3", process_step_3)

graph.set_entry_point("step1")
graph.add_edge("step1", "step2")
graph.add_edge("step2", "step3")
graph.add_edge("step3", END)

# Compile with memory checkpointer
checkpointer = MemorySaver()
app = graph.compile(checkpointer=checkpointer)

# Execute with thread tracking
thread_id = {"configurable": {"thread_id": "demo-thread"}}
initial_state = {"version": 0, "data": "Initial", "history": []}

print("=== Executing graph with checkpointing ===\n")
final_result = app.invoke(initial_state, thread_id)

print(f"Final version: {final_result['version']}")
print(f"Final data: {final_result['data']}")
print(f"History: {final_result['history']}\n")

# Retrieve checkpoint history - demonstrates immutability
print("=== Checkpoint History (each is an immutable snapshot) ===\n")
checkpoints = list(app.get_state_history(thread_id))

for i, checkpoint in enumerate(reversed(checkpoints)):
    state = checkpoint.values
    print(f"Checkpoint {i}:")
    print(f"  Version: {state.get('version', 'N/A')}")
    print(f"  Data: {state.get('data', 'N/A')}")
    print(f"  History: {state.get('history', [])}")
    print()

=== Executing graph with checkpointing ===

Final version: 3
Final data: Processed in step 3
History: ['Step 1 completed', 'Step 2 completed', 'Step 3 completed']

=== Checkpoint History (each is an immutable snapshot) ===

Checkpoint 0:
  Version: N/A
  Data: N/A
  History: []

Checkpoint 1:
  Version: 0
  Data: Initial
  History: []

Checkpoint 2:
  Version: 1
  Data: Processed in step 1
  History: ['Step 1 completed']

Checkpoint 3:
  Version: 2
  Data: Processed in step 2
  History: ['Step 1 completed', 'Step 2 completed']

Checkpoint 4:
  Version: 3
  Data: Processed in step 3
  History: ['Step 1 completed', 'Step 2 completed', 'Step 3 completed']



### **Example 5: Preventing Mutation Bugs**

In [4]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

class BuggyState(TypedDict):
    items: list[int]
    result: int

# BUGGY VERSION - Shows what happens with mutation
def buggy_node(state: BuggyState) -> dict:
    """
    This demonstrates a COMMON BUG - mutating state directly.
    DON'T DO THIS!
    """
    # BUG: Modifying state directly
    state["items"].append(999)  # This mutates the input!
    
    return {
        "result": sum(state["items"])
    }

# CORRECT VERSION - Immutable approach
def correct_node(state: BuggyState) -> dict:
    """
    Correct approach: create new list, don't modify original.
    """
    # CORRECT: Create new list with additional item
    new_items = state["items"] + [999]  # Creates new list
    
    return {
        "items": new_items,
        "result": sum(new_items)
    }

print("=== Demonstrating Mutation Bug ===\n")

# Test buggy version
original_list = [1, 2, 3]
buggy_state = {"items": original_list, "result": 0}

print(f"Original list before buggy_node: {original_list}")
buggy_result = buggy_node(buggy_state)
print(f"Original list AFTER buggy_node: {original_list}")
print(f"❌ BUG: Original list was mutated!\n")

# Test correct version
original_list_2 = [1, 2, 3]
correct_state = {"items": original_list_2, "result": 0}

print(f"Original list before correct_node: {original_list_2}")
correct_result = correct_node(correct_state)
print(f"Original list AFTER correct_node: {original_list_2}")
print(f"✅ CORRECT: Original list unchanged!")
print(f"New list in result: {correct_result['items']}")

print("\n=== Why This Matters in LangGraph ===")
print("""
When you mutate state directly:
1. Checkpoints can become inconsistent
2. Replaying execution gives different results
3. Concurrent nodes can interfere with each other
4. Debugging becomes extremely difficult
5. State history is corrupted

Always use immutable updates!
""")

=== Demonstrating Mutation Bug ===

Original list before buggy_node: [1, 2, 3]
Original list AFTER buggy_node: [1, 2, 3, 999]
❌ BUG: Original list was mutated!

Original list before correct_node: [1, 2, 3]
Original list AFTER correct_node: [1, 2, 3]
✅ CORRECT: Original list unchanged!
New list in result: [1, 2, 3, 999]

=== Why This Matters in LangGraph ===

When you mutate state directly:
1. Checkpoints can become inconsistent
2. Replaying execution gives different results
3. Concurrent nodes can interfere with each other
4. Debugging becomes extremely difficult
5. State history is corrupted

Always use immutable updates!



### **Example 6: Custom Immutable Reducer for Complex Merging**

In [5]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, END

# Custom immutable reducer for scores
def merge_scores_immutably(existing: dict, new: dict) -> dict:
    """
    Immutably merges score dictionaries.
    Takes max value for duplicate keys.
    """
    # Create new dictionary (immutable)
    merged = {**existing}  # Copy existing
    
    for key, value in new.items():
        if key in merged:
            # Take maximum score
            merged[key] = max(merged[key], value)
        else:
            merged[key] = value
    
    return merged  # Return new dictionary

# Custom immutable reducer for lists with deduplication
def append_unique_immutably(existing: list, new: list) -> list:
    """
    Immutably appends items, removing duplicates.
    """
    # Create new list with unique items
    combined = existing + new
    seen = set()
    result = []
    
    for item in combined:
        if item not in seen:
            seen.add(item)
            result.append(item)
    
    return result  # New list

class AnalysisState(TypedDict):
    scores: Annotated[dict, merge_scores_immutably]
    tags: Annotated[list[str], append_unique_immutably]
    final_score: float

def analyze_quality(state: AnalysisState) -> dict:
    """Analyze quality metrics."""
    return {
        "scores": {"quality": 0.85, "accuracy": 0.90},
        "tags": ["quality-check", "verified"]
    }

def analyze_performance(state: AnalysisState) -> dict:
    """Analyze performance metrics."""
    return {
        "scores": {"performance": 0.92, "quality": 0.88},  # quality will take max
        "tags": ["performance-check", "verified"]  # verified won't duplicate
    }

def compute_final_score(state: AnalysisState) -> dict:
    """Compute final aggregate score."""
    scores = state["scores"]
    avg_score = sum(scores.values()) / len(scores) if scores else 0.0
    
    return {
        "final_score": round(avg_score, 3)
    }

# Build graph
graph = StateGraph(AnalysisState)
graph.add_node("quality", analyze_quality)
graph.add_node("performance", analyze_performance)
graph.add_node("finalize", compute_final_score)

graph.set_entry_point("quality")
graph.add_edge("quality", "performance")
graph.add_edge("performance", "finalize")
graph.add_edge("finalize", END)

app = graph.compile()

# Execute
initial_state = {"scores": {}, "tags": [], "final_score": 0.0}

print("=== Demonstrating Custom Immutable Reducers ===\n")
print(f"Initial state: {initial_state}\n")

result = app.invoke(initial_state)

print("Final state:")
print(f"  Scores: {result['scores']}")
print(f"    (Note: quality=0.88, the max of 0.85 and 0.88)")
print(f"  Tags: {result['tags']}")
print(f"    (Note: 'verified' appears only once)")
print(f"  Final score: {result['final_score']}")

print(f"\nOriginal state unchanged: {initial_state}")

=== Demonstrating Custom Immutable Reducers ===

Initial state: {'scores': {}, 'tags': [], 'final_score': 0.0}

Final state:
  Scores: {'quality': 0.88, 'accuracy': 0.9, 'performance': 0.92}
    (Note: quality=0.88, the max of 0.85 and 0.88)
  Tags: ['quality-check', 'verified', 'performance-check']
    (Note: 'verified' appears only once)
  Final score: 0.9

Original state unchanged: {'scores': {}, 'tags': [], 'final_score': 0.0}


### **Example 7: Functional Purity and Reproducibility**

In [6]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

class PureState(TypedDict):
    input_value: int
    result: int
    call_count: int

def pure_transformation(state: PureState) -> dict:
    """
    Pure function: given same input, always returns same output.
    No side effects, no mutations, perfectly reproducible.
    """
    input_val = state["input_value"]
    
    # Deterministic computation
    result = (input_val * 2) + 10
    
    # Immutable update
    return {
        "result": result,
        "call_count": state["call_count"] + 1
    }

# Build simple graph
graph = StateGraph(PureState)
graph.add_node("transform", pure_transformation)
graph.set_entry_point("transform")
graph.add_edge("transform", END)

app = graph.compile()

print("=== Demonstrating Functional Purity & Reproducibility ===\n")

# Test reproducibility: same input → same output, every time
test_state = {"input_value": 5, "result": 0, "call_count": 0}

print("Running transformation 5 times with identical input:\n")

for i in range(1, 6):
    # Create fresh state each time (immutability in action)
    result = app.invoke(test_state)
    print(f"Run {i}: input={test_state['input_value']}, "
          f"result={result['result']}, call_count={result['call_count']}")

print("\n✅ Every run produces IDENTICAL results - perfect reproducibility!")

print("\n=== Verifying Immutability ===")
print(f"Original test_state: {test_state}")
print("❌ If state were mutable, test_state would have been modified!")
print("✅ But it's unchanged - immutability works!\n")

# Test determinism with different inputs
print("=== Testing Determinism with Different Inputs ===\n")
test_inputs = [1, 5, 10, 100]

for val in test_inputs:
    state = {"input_value": val, "result": 0, "call_count": 0}
    result = app.invoke(state)
    expected = (val * 2) + 10
    print(f"Input: {val:3d} → Result: {result['result']:3d} "
          f"(Expected: {expected:3d}) ✅")

print("\n💡 Key Insight:")
print("Because transformations are pure and immutable:")
print("  1. Results are 100% reproducible")
print("  2. Easy to test (just check input → output)")
print("  3. Can cache results safely")
print("  4. Can parallelize without fear")
print("  5. Debugging is straightforward")

=== Demonstrating Functional Purity & Reproducibility ===

Running transformation 5 times with identical input:

Run 1: input=5, result=20, call_count=1
Run 2: input=5, result=20, call_count=1
Run 3: input=5, result=20, call_count=1
Run 4: input=5, result=20, call_count=1
Run 5: input=5, result=20, call_count=1

✅ Every run produces IDENTICAL results - perfect reproducibility!

=== Verifying Immutability ===
Original test_state: {'input_value': 5, 'result': 0, 'call_count': 0}
❌ If state were mutable, test_state would have been modified!
✅ But it's unchanged - immutability works!

=== Testing Determinism with Different Inputs ===

Input:   1 → Result:  12 (Expected:  12) ✅
Input:   5 → Result:  20 (Expected:  20) ✅
Input:  10 → Result:  30 (Expected:  30) ✅
Input: 100 → Result: 210 (Expected: 210) ✅

💡 Key Insight:
Because transformations are pure and immutable:
  1. Results are 100% reproducible
  2. Easy to test (just check input → output)
  3. Can cache results safely
  4. Can paral

### **Key Takeaways**

1. **Immutable State Transitions = Reliability**  
   Never modify state in-place; always return new values.

2. **Use Reducers for Smart Merging**  
   Custom reducers enable complex, immutable state composition.

3. **Checkpoints are Atomic Snapshots**  
   Immutability ensures perfect checkpoint consistency and replay.

4. **Functional Purity Enables Testing**  
   Pure functions with immutable state are trivial to test and debug.

5. **Avoid Common Pitfalls**  
   - Don't use `.append()`, `+=`, or direct assignment
   - Create new lists/dicts with `+` or `{...}` spread operators
   - Use `copy.deepcopy()` for deeply nested structures when needed

6. **Performance is Not an Issue**  
   Structural sharing and shallow copies make immutability efficient.

7. **Immutability = Thread Safety**  
   Concurrent execution becomes safe by default.

---

### **Best Practices Summary**

| Do ✅                                      | Don't ❌                                 |
| ------------------------------------------ | ---------------------------------------- |
| `return {"list": state["list"] + [item]}` | `state["list"].append(item)`             |
| `return {"count": state["count"] + 1}`     | `state["count"] += 1`                    |
| `new_dict = {**old, "key": "new_value"}`   | `state["dict"]["key"] = "value"`         |
| Use custom reducers for complex merging    | Modify nested structures directly        |
| Treat state parameter as read-only         | Assume state modifications will persist  |

---

**Immutable state transitions are the foundation of reliable, scalable, and maintainable LangGraph applications.**